In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, label_binarize
from sklearn.metrics import classification_report, confusion_matrix 
from sklearn.metrics import ConfusionMatrixDisplay, roc_curve, auc

In [ ]:
names = ['Sepal Length', 'Sepal Width', 'Petal Length', 'Petal Width', 'target']
df = pd.read_csv('iris.data', names=names)
df.head()

In [ ]:
sns.countplot(df, x='target', hue='target', palette='viridis')
plt.show()

In [ ]:
X = df.drop(columns='target')
y = df['target']

X = StandardScaler().fit_transform(X)
y = LabelEncoder().fit_transform(y)

In [ ]:
def build_model(output_activation):
    model = Sequential([
        Input(shape=(4,)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(3, activation=output_activation)
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)
model_softmax = build_model('softmax')
history_softmax = model_softmax.fit(X_train, y_train, 
                                    validation_data=(X_test, y_test), 
                                    epochs=25, verbose=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)
model_sigmoid = build_model('sigmoid')
history_sigmoid = model_sigmoid.fit(X_train, y_train, 
                                    validation_data=(X_test, y_test), 
                                    epochs=25, verbose=1)

In [ ]:
plt.figure(figsize=(10,6))

plt.plot(history_softmax.history['accuracy'], label='Softmax - Train Accuracy', color='blue', linewidth=2)
plt.plot(history_softmax.history['val_accuracy'], label='Softmax - Val Accuracy', color='blue', linestyle='--')

plt.plot(history_sigmoid.history['accuracy'], label='Sigmoid - Train Accuracy', color='green', linewidth=2)
plt.plot(history_sigmoid.history['val_accuracy'], label='Sigmoid - Val Accuracy', color='green', linestyle='--')

plt.title('Training and Validation Accuracy Comparison')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
softmax_score = model_softmax.evaluate(X_test, y_test, verbose=0)
sigmoid_score = model_sigmoid.evaluate(X_test, y_test, verbose=0)

print(f"Test Accuracy with Softmax: {softmax_score[1]:.4f}")
print(f"Test Accuracy with Sigmoid: {sigmoid_score[1]:.4f}")

In [ ]:
plt.bar(['Softmax', 'Sigmoid'], [softmax_score[1], sigmoid_score[1]], color=['blue','green'])
plt.ylabel('Test Accuracy')
plt.title('Final Test Accuracy Comparison')
plt.ylim(0, 1)
plt.show()

In [ ]:
y_pred_softmax = np.argmax(model_softmax.predict(X_test), axis=1)
y_pred_sigmoid = np.argmax(model_sigmoid.predict(X_test), axis=1)

In [ ]:
labels = df['target'].unique()
print("Classification Report for Softmax:")
print(classification_report(y_test, y_pred_softmax, target_names=labels))

print("\nClassification Report for Sigmoid:")
print(classification_report(y_test, y_pred_sigmoid, target_names=labels))

In [ ]:
cm_softmax = confusion_matrix(y_test, y_pred_softmax)
cm_sigmoid = confusion_matrix(y_test, y_pred_sigmoid)

fig, ax = plt.subplots(1, 2, figsize=(14, 6))

ConfusionMatrixDisplay(cm_softmax, display_labels=labels).plot(ax=ax[0])
ax[0].set_title("Softmax Confusion Matrix", size=25)

ConfusionMatrixDisplay(cm_sigmoid, display_labels=labels).plot(ax=ax[1])
ax[1].set_title("Sigmoid Confusion Matrix", size=25)

plt.tight_layout()
plt.show()